In [1]:
import sqlalchemy
from sqlalchemy import create_engine, inspect
import mysql.connector
import psycopg2
import pandas as pd
import time
from config import host, user, user1, password, password1

In [2]:
start_time = time.time()

In [3]:
postgres_engine = create_engine(f'postgresql://{user1}:{password1}@{host}:5432/staging')

In [4]:
last_value = pd.read_sql('SELECT max(COUNTRY_ID) FROM stg_countries', con=postgres_engine).iloc[0,0]
last_value

203

In [5]:
### Establishing MySQL and PostgreSQL Connections

#MySQL Connection
try:
    mysql_db = mysql.connector.connect(host=host,user=user,password=password,db="eBooks2go")
    mycursor = mysql_db.cursor()
    print("MySQL Connection Established")
except mysql.connector.Error as e:
    print("Unable to Connect: ",format(e))

#PostgreSQL Connection
try:
    pg_db = psycopg2.connect(host=host,user=user1,password=password1,database="staging")
    pg_db.autocommit = True
    pgcursor = pg_db.cursor()
    print("PostgreSQL Connection Established")
except psycopg2.OperationalError as e:
    print("Unable to Connect: ",format(e))

MySQL Connection Established
PostgreSQL Connection Established


In [ ]:
count_success = 0
Total_Records_from_source = 0
count_failed_emails = 0

#Selects data from table
slt_cmd ="SELECT * FROM eBooks2go.BLC_COUNTRIES WHERE COUNTRY_ID > " + str(last_value)
mycursor.execute(slt_cmd)

#Extracting column names 
col_names_lst = [i[0] for i in mycursor.description] 
col_names = ', '.join(col_names_lst)+', Created_Date'+', Updated_Date'

#Inserting data from MySQL into PostgreSQL
placeholders = ', '.join(['%s'] * (len(col_names_lst)+2) ) 

#Inserting data into PostgreSQL
insrt_cmd = "INSERT INTO stg_COUNTRIES("+col_names+") " \
            "VALUES ( %s )" % (placeholders)

for row in mycursor:
    try:
        Total_Records_from_source = mycursor.rowcount
        pgcursor.execute(insrt_cmd,row+tuple([time.strftime("%Y/%m/%d %H:%M")])+tuple([time.strftime("%Y/%m/%d %H:%M")]))
        count_success += 1

    except Exception as e:
        print(e)
        count_failed_emails += 1

insrt_log = "INSERT INTO LOG_DIM (Phase, Source_Table_Name, Target_Table_Name, Total_Records_Source_Table, \
            Total_Records_Target_Table, Status, Remarks, Execution_time) Values ('LtoS','eBooks2go.BLC_COUNTRIES',\
            'stg_COUNTRIES'," +str(Total_Records_from_source)+", "+str(count_success)+",'Completed','"\
            +str(count_failed_emails)+" Failed','"+str(round(time.time() - start_time,2))+" seconds')"
pgcursor.execute(insrt_log)
     
        
mysql_db.close()
pg_db.close()